# 1. Set up

In [1]:
import os
import os.path as osp
import numpy as np
from IPython.display import display
from PIL import Image
import time
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm

In [5]:
import base64
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()

kaggle_cfg = {}
for name in ['enfit1_rsa', 'NGROK_TOKEN_1', 'ID_RSA_PUB', 'SSH_PASS']:
    try:
        kaggle_cfg[name] = user_secrets.get_secret(name)
    except:
        pass

enfit1_rsa_val = kaggle_cfg.get("enfit1_rsa", "")
if enfit1_rsa_val != "":
    enfit1_rsa_val = base64.b64decode(enfit1_rsa_val).decode('ascii')    
ngrok_token_val = kaggle_cfg.get("NGROK_TOKEN_1", "")
id_rsa_pub      = kaggle_cfg.get("ID_RSA_PUB", "")
ssh_pass_val    = kaggle_cfg.get("SSH_PASS", "12345")

# print(enfit1_rsa_val)
# print(ngrok_token_val)
# print(id_rsa_pub)
# print(ssh_pass_val)

In [6]:
![ ! -d "/kaggle/working/resyslab_utils" ] && git clone https://github.com/dntai/resyslab_utils.git /kaggle/working/resyslab_utils
![ -d "/kaggle/working/resyslab_utils" ] && cd /kaggle/working/resyslab_utils && git pull
%run /kaggle/working/resyslab_utils/resyslab_utils/clouds/cloud_setup.py

Cloning into '/kaggle/working/resyslab_utils'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 56 (delta 16), reused 50 (delta 10), pack-reused 0
Unpacking objects: 100% (56/56), 11.60 KiB | 742.00 KiB/s, done.
Already up to date.


In [7]:
start_ssh(id_rsa_pub=id_rsa_pub,
          install_ssh=True, 
          config_ssh=True, 
          password=ssh_pass_val)

********** SETUP SSH SERVICE **********
> Install ssh service...
> Copy public key to authorized keys...
> Config ssh service...
> Set root password...
> Restart SSH service...
 * Restarting OpenBSD Secure Shell server sshd
   ...done.

> Process ~/.bashrc to registry PS1, TERM...
PS1=
PS1='\[\e]0;\u@\h: \w\a\]\[\033\]\u@\h\[\033\]:\[\033\]\w\[\033\]$ '

---------- Finished ----------



In [8]:
start_ngrok([ngrok_token_val])

********** SETUP NGROK **********
> Install ngrok...
> Kill ngrok process...
/bin/bash: line 0: kill: `': not a pid or valid job spec
> Binding ports...
> Registry success!                                                                                 
ssh: NgrokTunnel: "tcp://2.tcp.ngrok.io:11960" -> "localhost:22"
vscode: NgrokTunnel: "https://7e43-34-105-114-238.ngrok-free.app" -> "http://localhost:9000"

---------- Finished ----------



In [9]:
setup_config_github(enfit1_rsa_val, 'enfit1_rsa', 'math')

********** CONFIG GITHUB **********
> Add id_rsa...
> Add config file...
> List ~/.ssh...
authorized_keys  config  enfit1_rsa
> Show config...
Host math
    HostName ssh.github.com
    User git
    Port 443
    StrictHostKeyChecking no
    IdentityFile ~/.ssh/enfit1_rsa
> Test ssh...
PTY allocation request failed on channel 0
Hi duongngockhanh! You've successfully authenticated, but GitHub does not provide shell access.
Connection to ssh.github.com closed.


In [11]:
!pip install wandb

In [12]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
wandb_api = user_secrets.get_secret("wandb_api") 

In [13]:
import wandb
wandb.login(key=wandb_api)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# 2. Train

In [10]:
!git clone math:duongngockhanh/colorful-image-colorization.git

Cloning into 'colorful-image-colorization'...
remote: Enumerating objects: 150, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 150 (delta 54), reused 101 (delta 24), pack-reused 0
Receiving objects: 100% (150/150), 2.13 MiB | 6.29 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [26]:
%run /kaggle/working/colorful-image-colorization/train.py

train_in_path = "/kaggle/input/small-coco-stuff/small-coco-stuff/train2017/train2017"
val_in_path = "/kaggle/input/small-coco-stuff/small-coco-stuff/train2017/train2017"
main(train_in_path=train_in_path, val_in_path=val_in_path)

EPOCH 1:	Train loss: 237.9828	Val loss: 349.8534
EPOCH 2:	Train loss: 247.4412	Val loss: 259.3879
EPOCH 3:	Train loss: 335.7952	Val loss: 340.4423
EPOCH 4:	Train loss: 304.7224	Val loss: 239.8981
EPOCH 5:	Train loss: 254.1961	Val loss: 237.2886
EPOCH 6:	Train loss: 239.6833	Val loss: 240.2247
EPOCH 7:	Train loss: 210.2900	Val loss: 290.9379
EPOCH 8:	Train loss: 233.6255	Val loss: 235.1957
EPOCH 9:	Train loss: 270.0838	Val loss: 231.8587
EPOCH 10:	Train loss: 293.6492	Val loss: 231.4199
EPOCH 11:	Train loss: 280.2869	Val loss: 237.1641
EPOCH 12:	Train loss: 239.4725	Val loss: 237.0736
EPOCH 13:	Train loss: 244.5599	Val loss: 227.3699
EPOCH 14:	Train loss: 218.5626	Val loss: 225.3013
EPOCH 15:	Train loss: 212.1948	Val loss: 225.9281
EPOCH 16:	Train loss: 215.3114	Val loss: 228.5491
EPOCH 17:	Train loss: 274.9720	Val loss: 226.9403
EPOCH 18:	Train loss: 295.6551	Val loss: 221.4300
EPOCH 19:	Train loss: 233.6291	Val loss: 232.4698
EPOCH 20:	Train loss: 270.2941	Val loss: 233.0336
EPOCH 21:

# 3. Git update

In [32]:
%cd colorful-image-colorization/
!git add .
!git commit -m "revise train: showed_image to device"
!git push
%cd ..

[Errno 2] No such file or directory: 'colorful-image-colorization/'
/kaggle/working/colorful-image-colorization
[main 5e7a5ae] revise train: showed_image to device
 1 file changed, 12 insertions(+), 10 deletions(-)
Enumerating objects: 5, done.
Counting objects: 100% (5/5), done.
Delta compression using up to 4 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 401 bytes | 401.00 KiB/s, done.
Total 3 (delta 2), reused 0 (delta 0)
remote: Resolving deltas: 100% (2/2), completed with 2 local objects.
To math:duongngockhanh/colorful-image-colorization.git
   0f899d1..5e7a5ae  main -> main
